In [12]:
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import os

In [14]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_api_key)

In [15]:
reader = PdfReader("me/SUKANTA BALA.pdf")
linkedin = ''
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
print(linkedin);



Sukanta Bala || Page  1 
  
 
 
SUMMARY  
Software developer  with professional experience  in building scalable , maintainable and secure  web applications 
using React .js, TypeScript , Vue.js , Node.js , and Python . Skilled in agile practices, OOP, CI/CD, and DevOps, 
with a track record of delivering performant web platforms and mentoring teams through code reviews.  
 
SKILLS  
o Frontend:  React.js , TypeScript , Vue.js, Tailwind CSS  
o Backend : Node.js , Express.js, Python, FastAPI, Microservices  
o Testing & Tooling : Vite, Jest, Linux,  Git, WebSockets , REST  API 
o DevOps : Docker , AWS , CI/CD, Nginx, Apache  
o Databases:  SQL, PostgreSQL , MySQL , MongoDB , Redis   
o AI technologies:  Generative AI, LLM’s concept , ChatGPT API , Copilot, Cursor.  
o Workflow:  Agile, Scrum, TDD , DDD  
EXPERIENCE  
Senior Full Stack Engineer   Oct 2023 – Aug, 2025 
Comworks Media | Sydney, Australia  (Remote)  
 Spearheading design and development of a scalable ERP system using Fast

In [16]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

print(summary)

Full Stack Engineer with 7+ years of experience in building scalable and maintainable applications, ensuring efficiency and reliability. Passionate about delivering scalable, high-performance solutions in Agile/Scrum environments. Seeking opportunities in European’s tech industry.


In [17]:
name = 'Sukanta Bala'

In [18]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

print(system_prompt)

You are acting as Sukanta Bala. You are answering questions on Sukanta Bala's website, particularly questions related to Sukanta Bala's career, background, skills and experience. Your responsibility is to represent Sukanta Bala for interactions on the website as faithfully as possible. You are given a summary of Sukanta Bala's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
Full Stack Engineer with 7+ years of experience in building scalable and maintainable applications, ensuring efficiency and reliability. Passionate about delivering scalable, high-performance solutions in Agile/Scrum environments. Seeking opportunities in European’s tech industry.

## LinkedIn Profile:
Sukanta Bala || Page  1 
  
 
 
SUMMARY  
Software developer  with professional experience  in building scalable , maintainable and

In [23]:
def chat(message, history):
    message = [{'role': 'system', 'content': system_prompt}] + history + [{'role': 'user', 'content': message}]
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=message,
    )
    return response.choices[0].message.content

In [24]:
gr.ChatInterface(chat, type='messages').launch(share=True)

* Running on local URL:  http://127.0.0.1:7864

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [25]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [26]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."
print(evaluator_system_prompt)

You are an evaluator that decides whether a response to a question is acceptable. You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. The Agent is playing the role of Sukanta Bala and is representing Sukanta Bala on their website. The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. The Agent has been provided with context on Sukanta Bala in the form of their summary and LinkedIn details. Here's the information:

## Summary:
Full Stack Engineer with 7+ years of experience in building scalable and maintainable applications, ensuring efficiency and reliability. Passionate about delivering scalable, high-performance solutions in Agile/Scrum environments. Seeking opportunities in European’s tech industry.

## LinkedIn Profile:
Sukanta Bala || Page  1 
  
 
 
SUMMARY  
Software developer  with profession

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_client = OpenAI(api_key=gemini_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [ ]:
def evaluate_response(reply, message, history):
    message = [{'role': 'system', 'content': evaluator_system_prompt}] + history + [{'role': 'user', 'content': evaluator_user_prompt(reply, message, history)}]
    response = gemini_client.beta.chat.completions.parse(
        model='gemini-2.0-flash',
        messages=message,
        response_format=Evaluation,
    )
    return response.choices[0].message.parsed
    

In [ ]:
message = [{'role': 'system', 'content': system_prompt}] + [{'role': 'user', 'content': 'Hello, how are you?'}]
response = openai_client.chat.completions.create(model='gpt-4o-mini', messages=message)
reply = response.choices[0].message.content